### Visualization of crack propagation in space and time

In [1]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN

Specify dilation factor

In [62]:
df = 3

In [63]:
node = np.genfromtxt("309555-e"+str(df)+"-node.txt",delimiter = ',')
link = np.genfromtxt("309555-e"+str(df)+"-link.txt",delimiter = ',')

In [64]:
numElem = link.shape[0]

Specify number of frames

In [65]:
numFrames = 70

Import status as numElem x 2 x numFrames array

In [66]:
statuses = np.genfromtxt("statuses/309555-e"+str(df)+"-tension-mises_status.txt",delimiter = ',',max_rows = numElem*numFrames)

In [67]:
statuses = np.transpose(np.reshape(statuses.T,(2, numElem, numFrames),order = 'F'),(1,0,2))

In [68]:
row_ind = np.argsort(statuses[:,0,:],0)

In [69]:
for i in range(0,statuses.shape[2]):
    sslice = statuses[:,:,i]
    sslice = sslice[row_ind[:,i]]
    statuses[:,:,i] = sslice

Calculate link centroids

In [70]:
centroids = np.zeros((link.shape[0],2))

In [71]:
for i in range(0,link.shape[0]):
    centroids[i,0] = (node[int(link[i,1]-1),1] + node[int(link[i,2]-1),1])/2
    centroids[i,1] = (node[int(link[i,1]-1),2] + node[int(link[i,2]-1),2])/2

Plot cracked points at time t

Find clusters with DBSCAN algorithm

In [78]:
fig, axes = plt.subplots(nrows=2, ncols=2)
axes = axes.flatten()
time = 58
for i in range(4):
    t = time+i
    ax = axes[i]
    crackpts = np.vstack((centroids[np.nonzero(statuses[:,1,t]==0)[0],0],centroids[np.nonzero(statuses[:,1,t]==0)[0],1])).T
    ax.set_title('t='+str(t))
    ax.scatter(centroids[:,0],centroids[:,1],color='yellow',zorder=0)
    if not crackpts.size:
        ax.scatter([],[])
    else:
        db = DBSCAN(eps=3, min_samples=10).fit(crackpts)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        unique_labels = set(labels)
        colors = [plt.cm.Paired(each)
                  for each in np.linspace(0, 1, len(unique_labels))]

        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]

            class_member_mask = (labels == k)

            xy = crackpts[class_member_mask & core_samples_mask]
            ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=7)

            xy = crackpts[class_member_mask & ~core_samples_mask]
            ax.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=3)
        
plt.suptitle('tension - erosion factor '+str(df))
plt.show()

FigureCanvasNbAgg()

3-D spacetime plots

In [662]:
t_start = 47
fig = plt.figure()
ax = fig.add_subplot(111,projection = '3d')
colors = [plt.cm.Paired(each)
            for each in np.linspace(0, 1, 5)]
for t in range(t_start,t_start+7):
    crackpts = np.vstack((centroids[np.nonzero(statuses[:,1,t]==0)[0],0],centroids[np.nonzero(statuses[:,1,t]==0)[0],1])).T
    if not crackpts.size:
        ax.scatter([],[])
    else:
        db = DBSCAN(eps=3, min_samples=10).fit(crackpts)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        unique_labels = set(labels)
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]

            class_member_mask = (labels == k)

            xy = crackpts[class_member_mask & core_samples_mask]
            ax.scatter(xy[:, 0], xy[:, 1],t*np.ones_like(xy[:,0]), c=tuple(col),
                     s=8)

            xy = crackpts[class_member_mask & ~core_samples_mask]
            ax.scatter(xy[:, 0], xy[:, 1],t*np.ones_like(xy[:,0]), c=tuple(col),
                     s=2)

    #     ax.scatter(crackpts[:,0],crackpts[:,1],t*np.ones_like(crackpts[:,0]))

ax.set_zlabel('time', rotation=90)
ax.set_zlim3d(t_start,t_start+6)
plt.show()
    

FigureCanvasNbAgg()

In [73]:
plt.close('all')